# TRAIN

In [ ]:
import pandas as pd
import os
import csv
import pickle
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "" # AÑADE AQUÍ TU PROPIA API KEY DE OPENAI
client = OpenAI()

# Cargar el archivo CSV
csv_file_path = 'https://docs.google.com/spreadsheets/d/1ILOeTEJ2qCf9A3dJXaGS1biDSeEyJ7UN/export?format=csv'
df = pd.read_csv(csv_file_path)

# Rutas de archivos
output_csv_path = 'descripciones_imagenes.csv'
error_csv_path = 'error_imagenes.csv'
checkpoint_path = 'checkpoint.pkl'

# Cargar errores existentes
if os.path.exists(error_csv_path):
    errores_df = pd.read_csv(error_csv_path, delimiter=';')
else:
    errores_df = pd.DataFrame(columns=["Nombre Carpeta", "Nombre Imagen", "Ruta Imagen", "Error"])

# Crear el archivo CSV si no existe y escribir los encabezados
if not os.path.exists(output_csv_path):
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(["Nombre Carpeta","Nombre Imagen", "Ruta Imagen", "Descripciones imagenes"])

# Cargar checkpoint
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        last_index = pickle.load(f)
else:
    last_index = -1  # Si no existe el checkpoint, empieza desde el principio

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Verificar si la imagen ya fue procesada o tuvo error
    esta_en_errores = (
        (errores_df["Nombre Carpeta"] == row["Nombre carpeta"]) &
        (errores_df["Nombre Imagen"] == row["Nombre imagen"]) &
        (errores_df["Ruta Imagen"] == row["Ruta imagen"])
    ).any()

    # Si no está en errores y ya pasó el checkpoint, saltar
    if index <= last_index and not esta_en_errores:
        continue

    if '/train' in row['Nombre carpeta']:
        nombre_carpeta = row['Nombre carpeta']
        tipo_de_clase, separacion, subcarpeta = row['Nombre carpeta'].split('/')
        nombre_imagen = row['Nombre imagen']
        ruta_imagen = row['Ruta imagen']

        print(ruta_imagen)

        try:
            # Enviar la solicitud a la API
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are a visual inspection expert specializing in quality control. You will be shown high-resolution images of physical products or components. "
                            "Your job is to carefully and thoroughly inspect each image for any visible issues, including but not limited to: cracks, scratches, missing parts, deformations, discoloration, dirt, misalignment, or any unusual thing you detect. "
                            "Carefully observe the entire image and describe all visible details in a single paragraph. List any anomalies with their type, location, and size if possible. "
                            "If no anomalies are present, don't state: 'No visible anomalies detected.' Only describe what is clearly visible. Be precise, objective, and avoid assumptions."
                        )
                    },
                    {
                        "role": "user",
                        "content": [
                            #{"type": "text", "text": "Inspect the image and provide a detailed description (maximum 300 characters)."},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"{ruta_imagen}"
                                },
                            },
                        ]
                    }
                ],
                max_tokens=300,
            )

            print(response.choices[0])

            # Extraer el contenido de la respuesta
            message_content = response.choices[0].message.content.strip() if response.choices else ""

            # Guardar la respuesta en el archivo CSV
            with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([nombre_carpeta, nombre_imagen, ruta_imagen, message_content])

            # Eliminar de errores si estaba registrado
            errores_df = errores_df[~(
                (errores_df["Nombre Carpeta"] == nombre_carpeta) &
                (errores_df["Nombre Imagen"] == nombre_imagen) &
                (errores_df["Ruta Imagen"] == ruta_imagen)
            )]

            # Reescribir el archivo de errores actualizado
            errores_df.to_csv(error_csv_path, sep=';', index=False)

            # Guardar checkpoint si es nuevo
            if index > last_index:
                with open(checkpoint_path, 'wb') as f:
                    pickle.dump(index, f)

            # Mensaje de confirmación
            print(f"Imagen {nombre_imagen} procesada y guardada en {output_csv_path}")

        except Exception as e:
            error_message = str(e)
            # Guardar los detalles de la imagen con error en el archivo CSV de errores
            with open(error_csv_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([nombre_carpeta, nombre_imagen, ruta_imagen, error_message])

            print(f"Error al procesar la imagen {nombre_imagen}: {e}")


# TEST

In [ ]:
ejemplos_por_tipo = {
    "breakfast_box": [
        "The image shows a divided container with two sections. The left section contains three pieces of fruit: two mandarin oranges and one peach. The mandarin oranges have a bright orange color with a few darker spots, while the peach has a reddish hue with some natural color variations. The right section contains a mixture of granola, almonds, and banana chips. The granola is light brown and looks crisp, with visible oats. The almonds are brown and placed among the banana chips, which are light tan with some small brownish spots. The container itself is white and appears to be made of a biodegradable material. The overall presentation is neat and orderly.",
        "The image shows a divided food container holding three whole fruits, possibly mandarins or tangerines, and a nectarine or peach on the left side. They appear fresh with usual surface texture, slight blemishes are visible on some of the fruits. On the right side, there is an assortment of granola, sliced dried bananas, and almonds. The granola covers the top half of the right section, with the dried bananas and almonds below it. The overall arrangement appears neat, with different elements placed separately within the confines of the tray. The container itself appears to be made of a biodegradable material.",
        "The image shows a divided tray containing three segments. The left segment contains two oranges, one on top of the other, and a peach placed in between them. The fruits appear fresh with the oranges having a vibrant orange color and slight texture details visible. The middle segment is filled with a generous portion of granola, consisting of oats and possibly some seeds. The granola has a golden-brown color, indicating it may be toasted. The right segment features banana chips and several whole almonds. The banana chips are light yellow with browning in the center, typical of dried bananas, while the almonds look crisp with a brown outer skin. The tray itself is off-white and has a simple design with rounded corners.",
        "The image shows a divided tray containing a selection of food items. On the left side, there are two oranges and a peach placed in a row. The peach has a yellowish-red skin with some red patches, while the oranges have a uniform bright orange color with some slight texture visible. On the right side of the tray, there is a mixture of granola, consisting of oats and possibly some seeds. Below the granola mix, there's a collection of dried banana chips and almonds. The granola and nuts are arranged neatly without any visible spills or irregularities. The tray itself appears to be a white, eco-friendly material and has compartments to separate the items. The presentation is clean and orderly.",
        "The image shows a compartmentalized tray containing three pieces of fruit on the left and a combination of granola, banana chips, and almonds on the right. The fruits on the left appear to be two mandarins and one peach or nectarine. The granola occupies the upper part of the right compartment, while the banana chips and almonds are towards the lower section. The tray is a simple, white, rectangular container, and the food items are neatly arranged without any visible signs of damage or spoilage. The almonds and banana chips appear uniform in size and texture."
    ],
    "juice_bottle": [
        "This image shows a clear glass bottle filled with a reddish-brown liquid, capped with a transparent lid. The bottle features two labels: one with an image of two cherries in red on a yellow background, and another stating '100% Juice' also on a yellow background. The labels appear to be slightly reflective. There are no visible deformities or anomalies on the bottle itself. The liquid inside is filled up to the shoulder of the bottle, with minimal air space visible. The glass appears clean and clear, with no noticeable scratches or cracks. The background is a dark, non-reflective surface that contrasts with the bottle.",
        "The image shows a clear glass bottle filled with a pale, creamy liquid. The bottle has a screw top and a smooth neck. There are two labels on the front of the bottle. The top label features an illustration of a bunch of bananas on a yellow background, and the bottom label reads '100% Juice' in black text on a yellow background. The labels appear to be securely attached and centered. The liquid inside appears consistent in color. The glass surface is reflective, and there are no visible scratches or signs of damage on the bottle.",
        "The image shows a glass bottle filled with a light yellow liquid. The bottle is sealed, with a transparent cap, and it has two labels on the front. The upper label displays an illustration of a fruit, possibly an orange, with a green leaf. The lower label reads '100% Juice' in a black font on a light-colored background. The bottle is positioned against a dark background, and the liquid level inside the bottle is consistent and appears to be approximately one inch below the neck. The glass is clear, and there are no visible cracks or scratches on the surface. There is a shadow on the right side of the bottle.",
        "The image shows a clear glass bottle filled with light yellow liquid, assumed to be juice. The bottle has a straight neck with threading at the top for a cap. A label on the front of the bottle features a graphic of an orange with a green leaf and another label at the bottom reads '100% Juice.' The bottle appears clean and smooth without visible scratches, cracks, or deformations. The labels are well-aligned and intact, and the liquid inside is consistent in color, with no visible particulates or stratification. The lighting creates a reflection on the surface of the bottle but does not obscure the contents or labels.",
        "The image depicts a small glass bottle filled with a dark red liquid. The liquid level is consistent, slightly below the neck of the bottle. The bottle has a narrow neck and a flat, wide body. Two labels are visible on the front: the upper label displays an illustration of two cherries on a yellow background, and the lower label reads '100% Juice' in black text on a yellow background. The surface of the glass appears clean and without scratches, and the labels are applied evenly without any noticeable wrinkles or tears. The overall appearance of the bottle suggests it is in good condition."
    ],
    "pushpins": [
        "The image shows an open plastic box with multiple compartments, each containing a yellow pushpin. The pins are randomly oriented within their respective compartments. The plastic box is transparent, allowing for visibility of the contents. The tips of the pushpins appear sharp and intact, with their yellow heads showing uniform color and shape. The compartments are evenly divided, and no major defects, such as cracks or discolorations in the plastic, are visible. The pins are distributed across the compartments with varying orientations, and there are no obvious missing pieces or irregularities.",
        "The image shows a plastic container divided into individual compartments, each holding a single yellow pushpin. The container is clear, allowing visibility of the contents, and each pushpin is aligned neatly in its compartment. The pushpins have a consistent design with a yellow plastic head and a sharp metal pin. There are no visible anomalies such as cracks, scratches, or dirt on the container or the pushpins. The compartments are evenly spaced, and the container lid appears intact and functional. Each pushpin is positioned differently but remains within its compartment, indicating no apparent damage or missing parts.",
        "The image shows a clear plastic container with multiple compartments, each containing yellow pushpins. The pushpins are arranged in various orientations within the compartments. The container appears to have four rows and four columns, making a total of 16 compartments. Each compartment holds one pushpin. The yellow pushpins are uniform in color, and their metal points are visible. The plastic container is transparent, allowing a clear view of the contents. There are no visible anomalies such as cracks or scratches on the container or the pushpins.",
        "The image shows a clear plastic box with multiple compartments, each containing a single yellow pushpin. The pushpins are evenly distributed across the compartments, positioned at various angles. The box appears to be intact with no visible cracks or damage. The pushpins have metallic pins that are also undamaged and properly secured to the plastic tops. The overall organization of the pushpins and the condition of both the box and pins appear to be in good order, with no visible signs of dirt or discoloration.",
        "The image shows a transparent plastic container with four compartments, each containing a yellow pushpin. The pushpins are randomly oriented, with some standing upright and others laying sideways. The container itself appears clean and intact, with no visible damage or scratches. Each compartment holds pushpins, distributed evenly, and there are no loose pins outside of the compartments. The pushpins themselves look uniform in color and shape, with no visible defects such as bent pins or broken parts. The compartments appear clear and free from dirt or debris."
    ],
    "screw_bag": [
        "The image shows a clear plastic zip-lock bag containing two metallic bolts, two washers, and two nuts. The bolts are aligned parallel to each other, showcasing their threaded shafts and hexagonal heads. The washers, one of which is beside the bolts, are circular with a noticeable split. The nuts are hexagonal and placed near the bottom of the bag, with visible markings, possibly indicating size or type. The bag has recycling symbols and markings along the opening edge. The components appear neatly arranged and undamaged within the bag.",
        "The image features a clear plastic zip-lock bag containing a set of metal fasteners, specifically two bolts, two nuts, and two washers. The bolts appear to be of uniform size with standard threading. The washers are circular with consistent diameter, and the nuts are hexagonal. The bag is sealed and has the recycling symbol 'PE-LD 04' printed repeatedly along the side. The fasteners are neatly arranged, and there are no visible scratches, dents, or rust. The bag itself shows no signs of tearing or damage, and the contents are clearly visible and intact.",
        "The image shows a clear plastic bag containing metallic hardware components, including two hex bolts, two nuts, and two washers. The hex bolts have a cylindrical head and threading along the shaft. The nuts are hexagonal with an internal thread. The washers are circular with a central hole. The bag features a printed recycling symbol and the code '04 PE-LD' along the right edge. The components appear evenly distributed and undamaged within the bag. The bag itself is sealed and in good condition with no visible tears or damage.",
        "The image shows a clear plastic bag containing metal hardware, specifically two bolts, two nuts, and two washers. The items are lying against a black background, and the bag features recycling symbols and the text '04 LD' on the right side. The bolts have a hexagonal head and are threaded, with one positioned horizontally and the other slightly tilted. The washers and nuts are placed nearby, aligned parallel to the bolts. There are no apparent scratches, dirt, or deformations on the metal components or the plastic bag. The bag appears sealed at the top with no visible damage or openings.",
        "The image shows a clear plastic bag containing metal hardware components. There are two bolts, two washers, and two nuts. All components appear metallic and are neatly arranged inside the bag. The bag has recycling symbols printed along one edge. The bolts have a consistent thread pattern, and the washers are circular, fitting neatly with the nuts and bolts. There are no visible defects or inconsistencies in the components from this view. The plastic bag is sealed and appears undamaged."
    ],
    "splicing_connectors": [
        "The image shows two transparent electrical connectors, each containing five bright orange levers. The connectors are aligned horizontally with a red wire connecting one side to the other through the wire slots. The background features a metallic mesh with a diamond pattern. The connectors and wire appear properly aligned, with no visible damage or misalignment. The mesh in the background is consistently patterned, with no noticeable defects.",
        "The image shows a short length of blue wire connected between two orange and clear plastic lever connectors. Each connector has three slots, and both connectors appear to be fully intact with the wire properly inserted and seated. The background consists of a metallic, diamond-patterned mesh. There are no visible anomalies such as cracks, misalignments, or discolorations on the connectors or the wire. The overall alignment of the setup seems to be correct and secure.",
        "The image shows two clear plastic connectors with orange levers, joined by a short yellow wire. The connectors are positioned against a diamond-patterned metal mesh background. Each connector appears to have two wire insertion points, one of which is used. The connectors and wire look intact with no visible damage or imperfections. The wire is neatly cut and fitted, showing printed text indicating specifications. The metal mesh provides a contrasting backdrop but has no apparent influence on the components.",
        "The image features two wire connectors with transparent bodies and orange levers, joined by a short length of yellow wire. The connectors are placed on a metallic mesh background. Each connector has two orange levers, which appear to be in the closed position. The yellow wire is neatly straight and connected securely between the connectors. There are no visible anomalies such as dirt, discoloration, or damage to the connectors or wire. The mesh serves purely as a background and is uniform in pattern.",
        "The image shows a metal mesh background with two clear plastic connectors containing orange components, connected by a single red wire. The connectors, each housing five orange levers, appear aligned and intact. The red wire is neatly inserted into the connector openings on the left and exits the corresponding openings on the right. There are no visible signs of wear or damage to the connectors, wire, or mesh. The background features a repetitive, diamond-shaped grid pattern, providing contrast to the bright orange and red components."
    ]
}

In [ ]:
import pandas as pd
import os
import csv
import pickle
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "" # AÑADE AQUÍ TU PROPIA API KEY DE OPENAI
client = OpenAI()

def construir_prompt_con_ejemplos(tipo, test_image_url, ejemplos_dict):
    ejemplos = ejemplos_dict.get(tipo, [])

    prompt_intro = (
        f"The following are normal, defect-free components of type '{tipo}':\n\n"
    )
    ejemplos_texto = "\n\n".join([f"Example {i+1}:\nDescription: {desc}" for i, desc in enumerate(ejemplos)])
    prompt_cierre = (
        "\n\nNow analyze the next image. Describe only the visual differences or irregularities compared to these normal examples. Do not describe common or expected features."
    )

    prompt_completo = prompt_intro + ejemplos_texto + prompt_cierre

    return [
        {"type": "text", "text": prompt_completo},
        {"type": "image_url", "image_url": {"url": test_image_url}},
    ]

# Cargar el archivo CSV
csv_file_path = 'https://docs.google.com/spreadsheets/d/1ILOeTEJ2qCf9A3dJXaGS1biDSeEyJ7UN/export?format=csv'
df = pd.read_csv(csv_file_path)

# Rutas de archivos
output_csv_path = 'descripciones_imagenes_test.csv'
error_csv_path = 'error_imagenes_test.csv'
checkpoint_path = 'checkpoint.pkl'

# Cargar errores existentes
if os.path.exists(error_csv_path):
    errores_df = pd.read_csv(error_csv_path, delimiter=';')
else:
    errores_df = pd.DataFrame(columns=["Nombre Carpeta", "Nombre Imagen", "Ruta Imagen", "Error"])
    errores_df.to_csv(error_csv_path, sep=';', index=False)

# Crear el archivo CSV si no existe y escribir los encabezados
if not os.path.exists(output_csv_path):
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(["Nombre Carpeta","Nombre Imagen", "Ruta Imagen", "Descripciones imagenes"])

# Cargar checkpoint
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        last_index = pickle.load(f)
else:
    last_index = -1  # Si no existe el checkpoint, empieza desde el principio

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Verificar si la imagen ya fue procesada o tuvo error
    esta_en_errores = (
        (errores_df["Nombre Carpeta"] == row["Nombre carpeta"]) &
        (errores_df["Nombre Imagen"] == row["Nombre imagen"]) &
        (errores_df["Ruta Imagen"] == row["Ruta imagen"])
    ).any()

    # Si no está en errores y ya pasó el checkpoint, saltar
    if index <= last_index and not esta_en_errores:
        continue

    if '/test' in row['Nombre carpeta']:
        nombre_carpeta = row['Nombre carpeta']
        tipo_de_clase, separacion, subcarpeta = row['Nombre carpeta'].split('/')
        nombre_imagen = row['Nombre imagen']
        ruta_imagen = row['Ruta imagen']

        print(ruta_imagen)

        try:
            # Obtener tipo desde nombre_carpeta
            tipo = nombre_carpeta.split('/')[0]

            # Construir mensaje con ejemplos de ese tipo
            user_prompt = construir_prompt_con_ejemplos(tipo, ruta_imagen, ejemplos_por_tipo)

            # API request
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": (
                        "You are a visual inspection expert. You will be shown a series of normal (defect-free) component descriptions, "
                        "and then a new image to inspect.\n"
                        "Your task is to compare the new image against the normal examples and describe only visual differences or anomalies. "
                        "Do not mention features that are similar or expected. "
                        "Do not include line breaks. Write a single, concise paragraph highlighting only unusual or defective elements."
                    )},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=300,
            )

            print(response.choices[0])

            # Extraer el contenido de la respuesta
            message_content = response.choices[0].message.content.strip() if response.choices else ""

            # Guardar la respuesta en el archivo CSV
            with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([nombre_carpeta, nombre_imagen, ruta_imagen, message_content])

            # Eliminar de errores si estaba registrado
            errores_df = errores_df[~(
                (errores_df["Nombre Carpeta"] == nombre_carpeta) &
                (errores_df["Nombre Imagen"] == nombre_imagen) &
                (errores_df["Ruta Imagen"] == ruta_imagen)
            )]

            # Reescribir el archivo de errores actualizado
            errores_df.to_csv(error_csv_path, sep=';', index=False)

            # Guardar checkpoint si es nuevo
            if index > last_index:
                with open(checkpoint_path, 'wb') as f:
                    pickle.dump(index, f)

            # Mensaje de confirmación
            print(f"Imagen {nombre_imagen} procesada y guardada en {output_csv_path}")

        except Exception as e:
            error_message = str(e)
            # Guardar los detalles de la imagen con error en el archivo CSV de errores
            with open(error_csv_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([nombre_carpeta, nombre_imagen, ruta_imagen, error_message])

            print(f"Error al procesar la imagen {nombre_imagen}: {e}")
